In [1]:
# CÓDIGO ORIGINAL

import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c 


float64
8.11 ms ± 53.9 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.3 ms ± 4.58 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.5 ms ± 171 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [ ]:
# 1. LIBRERÍA CUPY

# Preparación para GPU
import cupy as cp
from cupyx.profiler import benchmark

# Pasamos a float32 (más realista para GPU)
a_cpu_32 = a_cpu.astype(np.float32)
b_cpu_32 = b_cpu.astype(np.float32)

In [9]:
# 1A) CUPY CON COPIA
def cupy_con_copia(x_cpu, y_cpu, a, b, c):
    # Copia CPU → GPU
    x_gpu = cp.asarray(x_cpu)
    y_gpu = cp.asarray(y_cpu)
    
    # Cálculo en GPU (ufuncs)
    z_gpu = a * x_gpu**2 + b * y_gpu + c
    
    # Copia GPU → CPU
    return cp.asnumpy(z_gpu)

print("CuPy CON copia (CPU↔GPU + cálculo):")
print(benchmark(
    cupy_con_copia,
    (a_cpu_32, b_cpu_32, a, b, c),
    n_repeat=5
))


CuPy CON copia (CPU↔GPU + cálculo):
cupy_con_copia      :    CPU:  7723.900 us   +/- 38.556 (min:  7685.202 / max:  7791.935) us     GPU-0:  7728.864 us   +/- 38.642 (min:  7689.888 / max:  7796.576) us


In [3]:
# 1B) CUPY SIN COPIA
a_gpu_32 = cp.asarray(a_cpu_32)
b_gpu_32 = cp.asarray(b_cpu_32)

def cupy_sin_copia(x_gpu, y_gpu, a, b, c):
    return a * x_gpu**2 + b * y_gpu + c


print("\nCuPy SIN copia (solo cálculo en GPU):")
print(benchmark(
    cupy_sin_copia,
    (a_gpu_32, b_gpu_32, a, b, c),
    n_repeat=5
))


CuPy SIN copia (solo cálculo en GPU):
cupy_sin_copia      :    CPU:    97.619 us   +/-  1.531 (min:    95.587 / max:    99.491) us     GPU-0:   904.064 us   +/-  1.213 (min:   902.144 / max:   905.216) us


In [4]:
# 2. LIBRERÍA NUMBA CUDA

from numba import vectorize, float32

@vectorize([float32(float32, float32, float32, float32, float32)],
           target='cuda')
def grade2_vector_cuda(x, y, a, b, c):
    return a * x * x + b * y + c

a_cpu_32 = a_cpu.astype(np.float32)
b_cpu_32 = b_cpu.astype(np.float32)


In [6]:
# 2A) NUMBA CUDA CON COPIA
from cupyx.profiler import benchmark

def numba_con_copia(x_cpu, y_cpu, a, b, c):
    return grade2_vector_cuda(x_cpu, y_cpu, a, b, c)

print("Numba CUDA CON copia:")
print(benchmark(
    numba_con_copia,
    (a_cpu_32, b_cpu_32, a, b, c),
    n_repeat=5
))


Numba CUDA CON copia:
numba_con_copia     :    CPU:  9647.698 us   +/- 1658.731 (min:  7485.653 / max: 11543.998) us     GPU-0:  9655.827 us   +/- 1659.743 (min:  7492.384 / max: 11552.000) us


In [7]:
# 2B) NUMBA CUDA SIN COPIA
a_gpu_32 = cp.asarray(a_cpu_32)
b_gpu_32 = cp.asarray(b_cpu_32)

def numba_sin_copia(x_gpu, y_gpu, a, b, c):
    return grade2_vector_cuda(x_gpu, y_gpu, a, b, c)

print("\nNumba CUDA SIN copia:")
print(benchmark(
    numba_sin_copia,
    (a_gpu_32, b_gpu_32, a, b, c),
    n_repeat=5
))



Numba CUDA SIN copia:
numba_sin_copia     :    CPU:  1512.934 us   +/- 650.415 (min:   931.589 / max:  2313.732) us     GPU-0:  1630.355 us   +/- 560.082 (min:  1127.424 / max:  2320.544) us


# ANÁLISIS DE LOS RESULTADOS
Este ejercicio permite estudiar la ejecución del cálculo de segundo grado usando CPU, CuPy y Numba CUDA, tanto copiando los datos entre CPU y GPU como manteniéndolos en la memoria de la GPU.

Como referencia en CPU, la implementación con Numba (@njit) presenta el mejor rendimiento, con tiempos en torno a 8 ms, mientras que las versiones basadas en ufuncs de NumPy se sitúan alrededor de 18 ms.

En el caso de CuPy con copia de datos, el tiempo total se mantiene en el orden de 7–8 ms, similar al de la ejecución en CPU. Esto indica que el coste de transferencia de datos entre CPU y GPU domina el tiempo total y anula las ventajas del paralelismo de la GPU para este tipo de operación.

Cuando los datos se mantienen en la memoria de la GPU, el tiempo reportado en CPU (del orden de 100 μs) corresponde principalmente al lanzamiento del kernel y a la gestión de la ejecución asíncrona, mientras que el tiempo reportado en GPU (del orden de 0.9 ms) representa el tiempo real de cálculo del vector en la GPU. Este último debe compararse con el tiempo de cálculo en CPU, que es del orden de 8 ms. Esto pone de manifiesto que el cálculo en sí es significativamente más rápido en GPU que en CPU. 

Resultados similares se observan con Numba CUDA. Cuando la copia de datos se realiza implícitamente en cada llamada, los tiempos se sitúan en torno a 9–10 ms, nuevamente ralentizados por la transferencia de memoria. Sin embargo, cuando los datos ya están en la GPU, el tiempo de ejecución se reduce a aproximadamente 1.5 ms, mostrando una mejora clara frente a la CPU, aunque ligeramente inferior a la obtenida con CuPy debido a que los kernels generados por Numba son menos optimizados.

En conjunto, estos resultados muestran que el uso de GPU solo resulta beneficioso cuando se minimizan las transferencias de datos entre CPU y GPU, y que CuPy ofrece el mejor rendimiento cuando se trabaja con datos ya en la GPU, mientras que Numba proporciona una alternativa flexible con un coste ligeramente superior.